# <font color='darkred'> Clustering sur les épisodes

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.feature_extraction import stop_words
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import math
from scipy import sparse

In [2]:
def verifier_ligne(ligne):
    """
    return True si la ligne est un sous-titre, False sinon
    """
    timestamp_regex = r'[0-9]{2}:[0-9]{2}:[0-9]{2}' 
    subnumber_regex =r'^[0-9]+$'
    liste_regex = [timestamp_regex, subnumber_regex]

    for regex in liste_regex:
        if re.match(regex, ligne):
            return False
    return True

def preprocessing_fichier(nom_fichier) :
    """
    string -> string
    à partir du nom d'un fichier de sous-titres, renvoie le texte des sous-titres
    """
    fichier = open(nom_fichier, 'r', encoding = "ISO-8859-1")
    lignes = fichier.readlines()
    fichier.close()
    texte = ""
    for ligne in lignes :
        if verifier_ligne(ligne) :

            m = re.sub(r'[^\w'+"-"']', ' ', ligne)
            
            texte += m
    
    return texte


def scan_folder(parent_folder, corp):
    """
    retourne corpus des textes contenus dans parent_folder sous forme de liste de string
    """
    # iterate over all the files in directory 'parent_folder'
    for file_name in os.listdir(parent_folder):
        if file_name.endswith(".txt"):
            path = parent_folder+"/"+file_name
            
            texte = preprocessing_fichier(path)
            
            corp.append(texte)
        
        else:
            current_path = "".join((parent_folder, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recall this method
                scan_folder(current_path, corp)
    
    return corp


#pour eviter les variables globales
def get_corpus(parent_folder):
    c = []
    res = scan_folder(parent_folder, c)
    return res


def get_hist(df, x_axis, y_axis, titre, colour, font_size=None, horizontal=False):
    """
    
    """
    if horizontal:
        hist = df.plot.barh(x=x_axis, y=y_axis, color=colour, title =titre, fontsize = font_size, edgecolor = "none").get_figure()
    else:
        hist = df.plot.bar(x=x_axis, y=y_axis, color=colour, title =titre, fontsize = font_size, edgecolor = "none").get_figure()
    path_fig = "img/"+titre+'.png'
    hist.savefig(path_fig,  bbox_inches="tight")

## <font color='darkblue'> Clustering par saison

On veut voir si l'algo des kMeans est capable de différencier les épisodes de la saison 1 d'Avatar de la saison 2 de la même série en se basant sur les tf-idf des mots par rapport aux deux saisons.

<br></br>
<font color='red'>Matrice des tf-idf : par rapport à la saison de référence ou par rapport aux deux saisons ?</font>

In [10]:
corpus = get_corpus("/Vrac/PLDAC_addic7ed/data/175___Avatar__The_Last_Airbender/01") # liste des épisodes
corpus += get_corpus("/Vrac/PLDAC_addic7ed/data/175___Avatar__The_Last_Airbender/02")
#definition de l'ensemble de stopwords
nltk_sw = set(stopwords.words('english'))
sklearn_sw = set(stop_words.ENGLISH_STOP_WORDS)
stopwords_set = nltk_sw | sklearn_sw
l_nb = [str(i) for i in range(1000000)]
l_mots = ["don", "yeah", "hey", "okay", "oh", "uh", "yes", "ok"]
for mot in l_mots :
    stopwords_set.add(mot)
for nb in l_nb:
    stopwords_set.add(nb)

In [11]:
vectorizer = TfidfVectorizer(stop_words = stopwords_set)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dico = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)

In [23]:
# labels corrects pour calculer l'accuracy
# 1 = saison 1
# 0 = saison 2

y = []
for i in range(0, 21) :
    y.append(1)
for i in range(0, 20) :
    y.append(0)
len(y)

41

In [20]:
matrix_tfidf = df_tfidf.values
# pour la transformer en sparse matrix
sparse_matrix_tfidf = sparse.csr_matrix(matrix_tfidf)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(sparse_matrix_tfidf)
labels = kmeans.labels_

In [24]:
cpt = 0
for i in range(0, len(labels)) :
    if y[i] == labels[i] :
        cpt += 1
print("Accuracy = "+str(cpt/len(labels)))

Accuracy = 0.8048780487804879


Pour cet exemple en particulier, l'accuracy est très bonne.

## <font color='darkblue'> Clustering par série

On veut voir la même chose qu'avant mais cette fois-ci entre les épisodes de deux séries différentes.

### <font color='darkgreen'> Séries très différentes

Premiers tests sur deux séries très différentes :
- Lost
- Avatar

On s'attend à de bons résultats : le vocabulaire employé dans les deux séries est très différent.

In [51]:
corpus1 = get_corpus("/Vrac/PLDAC_addic7ed/data/175___Avatar__The_Last_Airbender/01") # liste des épisodes
corpus2 = get_corpus("/Vrac/PLDAC_addic7ed/data/1___Lost/01")
corpus = corpus1 + corpus2

In [52]:
y = []
for i in range(0, len(corpus1)) :
    y.append(0)
for i in range(0, len(corpus2)) :
    y.append(1)
len(y)

45

In [53]:
vectorizer = TfidfVectorizer(stop_words = stopwords_set)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dico = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)

In [54]:
matrix_tfidf = df_tfidf.values
# pour la transformer en sparse matrix
sparse_matrix_tfidf = sparse.csr_matrix(matrix_tfidf)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(sparse_matrix_tfidf)
labels = kmeans.labels_

In [55]:
for i in range(0, len(labels)) :
    print("Épisode "+str(i)+" classé dans le groupe : "+str(labels[i]))

Épisode 0 classé dans le groupe : 0
Épisode 1 classé dans le groupe : 0
Épisode 2 classé dans le groupe : 0
Épisode 3 classé dans le groupe : 0
Épisode 4 classé dans le groupe : 0
Épisode 5 classé dans le groupe : 0
Épisode 6 classé dans le groupe : 0
Épisode 7 classé dans le groupe : 0
Épisode 8 classé dans le groupe : 0
Épisode 9 classé dans le groupe : 0
Épisode 10 classé dans le groupe : 0
Épisode 11 classé dans le groupe : 0
Épisode 12 classé dans le groupe : 0
Épisode 13 classé dans le groupe : 0
Épisode 14 classé dans le groupe : 0
Épisode 15 classé dans le groupe : 0
Épisode 16 classé dans le groupe : 0
Épisode 17 classé dans le groupe : 0
Épisode 18 classé dans le groupe : 0
Épisode 19 classé dans le groupe : 0
Épisode 20 classé dans le groupe : 0
Épisode 21 classé dans le groupe : 1
Épisode 22 classé dans le groupe : 1
Épisode 23 classé dans le groupe : 1
Épisode 24 classé dans le groupe : 1
Épisode 25 classé dans le groupe : 1
Épisode 26 classé dans le groupe : 1
Épisode 27 

In [56]:
cpt = 0
for i in range(0, len(labels)) :
    if y[i] == labels[i] :
        cpt += 1
print("Accuracy = "+str(cpt/len(labels)))

Accuracy = 1.0


Pour ces deux séries, l'algorithme des kMeans avoir parfaitement à diviser les deux groupes par rapport aux tf-idf des mots présents dans les épisodes.

### <font color='darkgreen'> Séries très similaires

On s'intéresse maintenant à deux séries avec un vocabulaire plus similaire.
- Dr House
- Grey's Anatomy



In [46]:
path_serie1 ="/Vrac/PLDAC_addic7ed/data/30___Grey_s_Anatomy/01"
path_serie2 = "/Vrac/PLDAC_addic7ed/addic7ed/15___House/01"

In [57]:
corpus1 = get_corpus(path_serie1)
corpus2 = get_corpus(path_serie2)
corpus = corpus1 + corpus2

In [62]:
y = []
for i in range(0, len(corpus1)) :
    y.append(1)
for i in range(0, len(corpus2)) :
    y.append(0)
len(y)

31

In [59]:
vectorizer = TfidfVectorizer(stop_words = stopwords_set)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dico = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)

In [60]:
matrix_tfidf = df_tfidf.values
# pour la transformer en sparse matrix
sparse_matrix_tfidf = sparse.csr_matrix(matrix_tfidf)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(sparse_matrix_tfidf)
labels = kmeans.labels_
len(labels)

31

In [61]:
for i in range(0, len(labels)) :
    print("Épisode "+str(i)+" classé dans le groupe : "+str(labels[i]))

Épisode 0 classé dans le groupe : 1
Épisode 1 classé dans le groupe : 1
Épisode 2 classé dans le groupe : 1
Épisode 3 classé dans le groupe : 1
Épisode 4 classé dans le groupe : 1
Épisode 5 classé dans le groupe : 1
Épisode 6 classé dans le groupe : 1
Épisode 7 classé dans le groupe : 1
Épisode 8 classé dans le groupe : 1
Épisode 9 classé dans le groupe : 0
Épisode 10 classé dans le groupe : 0
Épisode 11 classé dans le groupe : 0
Épisode 12 classé dans le groupe : 0
Épisode 13 classé dans le groupe : 0
Épisode 14 classé dans le groupe : 0
Épisode 15 classé dans le groupe : 0
Épisode 16 classé dans le groupe : 0
Épisode 17 classé dans le groupe : 0
Épisode 18 classé dans le groupe : 0
Épisode 19 classé dans le groupe : 0
Épisode 20 classé dans le groupe : 0
Épisode 21 classé dans le groupe : 0
Épisode 22 classé dans le groupe : 0
Épisode 23 classé dans le groupe : 0
Épisode 24 classé dans le groupe : 0
Épisode 25 classé dans le groupe : 0
Épisode 26 classé dans le groupe : 0
Épisode 27 

In [63]:
cpt = 0
for i in range(0, len(labels)) :
    if y[i] == labels[i] :
        cpt += 1
print("Accuracy = "+str(cpt/len(labels)))

Accuracy = 1.0


Toujours une accuracy de 1. Sûrement du à la présence des noms propres qui doivent permettre une bonne discrimination des épisodes.

## <font color='darkblue'> Problèmes des noms propres

Afin de pimenter un peu la classification, on va essayer d'enlever les noms propres, qui sont très discriminants pour différencier deux épisodes de séries différentes.

In [88]:
path_serie1 ="/Vrac/PLDAC_addic7ed/data/30___Grey_s_Anatomy/01"
path_serie2 = "/Vrac/PLDAC_addic7ed/addic7ed/15___House/01"
corpus1 = get_corpus(path_serie1)
corpus2 = get_corpus(path_serie2)
corpus = corpus1 + corpus2

In [ ]:
y = []
for i in range(0, len(corpus1)) :
    y.append(1)
for i in range(0, len(corpus2)) :
    y.append(0)
len(y)

In [89]:
vectorizer = TfidfVectorizer(stop_words = stopwords_set)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dico = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)

In [90]:
nlargest = 20
order = np.argsort(-df_tfidf.values, axis=1)[:, :nlargest]
result = pd.DataFrame(df_tfidf.columns[order], 
                      columns=['top{}'.format(i) for i in range(1, nlargest+1)],
                      index=df_tfidf.index)
result.tail()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20
26,james,twitch,tuberculoma,whoah,wrist,know,victoria,mri,homeless,pin,foreman,fury,meningitis,like,got,streets,terharg,prozac,right,treatment
27,mark,attack,alzheimer,stacy,paris,yoga,want,think,blah,aip,know,right,gregg,like,going,encephalitis,warner,mountain,paranoia,house
28,meningitis,know,need,rash,going,right,like,form,neck,mary,blue,ow,bleeding,interview,blood,area,interviewing,brenda,puncture,old
29,dvt,schizophrenia,mom,vitamin,luke,know,old,gonna,birthday,right,think,ampicillin,pickles,schizophrenic,pt,young,dr,blood,shrink,praise
30,matt,chi,hydrolase,kid,davis,poisoned,know,son,time,clothes,cdc,tomato,got,poison,bus,washed,pesticides,mom,make,detergent


On voit que les tf-idf les plus élevés sont très souvent des noms propres.

Le but va être de les éliminer pour chaque épisode afin de voir si les kMeans peuvent différencier sur autre chose.

Dans un premier temps, on enlève, pour chaque épisode, les 20 mots dont les tf-idf sont les plus grands. C'est peut-être un peu trop, on ajustera après.

Il faut soit remplacer le tf-idf de ces mots par autre chose ou bien les enlever du vocabulaire pour tous les épisodes.
- remplacer : par un 0 ? un nombre aléatoire ? autre chose ?
        -> on veut que ces mots ne soient plus discriminants pour la classification
- enlever : on récupère pour chaque épisode les 20 mots avec les plus grands tf-idf et on crée un ensemble puis on enlève tous ces mots dans tous les épisodes 

### <font color='darkgreen'> Élimination du dataFrame

On récupère pour chaque épisode les 20 mots avec les plus grands tf-idf et on crée un ensemble puis on enlève tous ces mots dans tous les épisodes.

In [92]:
noms = result.values
set_noms_propres = set()
for ep in noms :
    for mot in ep :
        set_noms_propres.add(mot)
liste_noms_propres = list(set_noms_propres)
len(liste_noms_propres)

394

In [93]:
df_tfidf.head()

,00,000,007,00am,03,100l65,100mg,11th,150s,16th,...,yummy,yup,zebra,zebras,zero,zeros,zit,zits,zoo,ªclickâ
0,0.000000,0.000000,0.000000,0.000000,0.0,0.020857,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.059528,0.000000,0.000000,0.018101,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.016155,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.011254,0.175171,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.012743,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.058782,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
df_copy = df_tfidf.copy()


In [116]:
# pour se débarasser des colonnes
for mot in liste_noms_propres :
    df_copy = df_copy.drop(mot, axis=1)
    
df_copy.head()

,00,000,00am,03,100l65,100mg,11th,150s,16th,1980s,...,yummy,yup,zebra,zebras,zero,zeros,zit,zits,zoo,ªclickâ
0,0.000000,0.000000,0.000000,0.0,0.020857,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.059528,0.000000,0.018101,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.016155,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.011254,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.012743,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.058782,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
vectorizer = TfidfVectorizer(stop_words = stopwords_set)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dico = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)

In [110]:
y = []
for i in range(0, len(corpus1)) :
    y.append(1)
for i in range(0, len(corpus2)) :
    y.append(0)
len(y)

31

In [108]:
matrix_tfidf = df_tfidf.values
# pour la transformer en sparse matrix
sparse_matrix_tfidf = sparse.csr_matrix(matrix_tfidf)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(sparse_matrix_tfidf)
labels = kmeans.labels_
len(labels)

31

In [109]:
for i in range(0, len(labels)) :
    print("Épisode "+str(i)+" classé dans le groupe : "+str(labels[i]))

Épisode 0 classé dans le groupe : 1
Épisode 1 classé dans le groupe : 1
Épisode 2 classé dans le groupe : 1
Épisode 3 classé dans le groupe : 1
Épisode 4 classé dans le groupe : 1
Épisode 5 classé dans le groupe : 1
Épisode 6 classé dans le groupe : 1
Épisode 7 classé dans le groupe : 1
Épisode 8 classé dans le groupe : 1
Épisode 9 classé dans le groupe : 0
Épisode 10 classé dans le groupe : 0
Épisode 11 classé dans le groupe : 0
Épisode 12 classé dans le groupe : 0
Épisode 13 classé dans le groupe : 0
Épisode 14 classé dans le groupe : 0
Épisode 15 classé dans le groupe : 0
Épisode 16 classé dans le groupe : 0
Épisode 17 classé dans le groupe : 0
Épisode 18 classé dans le groupe : 0
Épisode 19 classé dans le groupe : 0
Épisode 20 classé dans le groupe : 0
Épisode 21 classé dans le groupe : 0
Épisode 22 classé dans le groupe : 0
Épisode 23 classé dans le groupe : 0
Épisode 24 classé dans le groupe : 0
Épisode 25 classé dans le groupe : 0
Épisode 26 classé dans le groupe : 0
Épisode 27 

In [111]:
cpt = 0
for i in range(0, len(labels)) :
    if y[i] == labels[i] :
        cpt += 1
print("Accuracy = "+str(cpt/len(labels)))

Accuracy = 1.0


toujours une accuracy de 1 :o

## <font color='darkblue'> Des petites fonctions propres

In [114]:
def classif_deux_series(path_serie1, path_serie2, stopwords_set) :
    """
    pour classifier deux séries
    """
    path_serie1 ="/Vrac/PLDAC_addic7ed/data/30___Grey_s_Anatomy/01"
    path_serie2 = "/Vrac/PLDAC_addic7ed/addic7ed/15___House/01"
    corpus1 = get_corpus(path_serie1)
    corpus2 = get_corpus(path_serie2)
    corpus = corpus1 + corpus2
    
    
    vectorizer = TfidfVectorizer(stop_words = stopwords_set)
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    dico = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df_tfidf = pd.DataFrame(denselist, columns=feature_names)
    
    
    matrix_tfidf = df_tfidf.values
    # pour la transformer en sparse matrix
    sparse_matrix_tfidf = sparse.csr_matrix(matrix_tfidf)
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=2, random_state=0).fit(sparse_matrix_tfidf)
    
    return kmeans.labels_

In [113]:
def accuracy(labels, y) :
    """
    labels : labels prédis 
    y : labels corrects
    renvoie l'accuracy
    """

    cpt = 0
    for i in range(0, len(labels)) :
        if y[i] == labels[i] :
            cpt += 1
    
    return cpt/len(labels)